In [53]:
import torch
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

from concept_erasure import LeaceEraser

n, d, k = 2048, 128, 2

X, Y = make_classification(
    n_samples=n,
    n_features=d,
    n_classes=k,
    random_state=42,
)
X_t = torch.from_numpy(X)
Y_t = torch.from_numpy(Y)

# Logistic regression does learn something before concept erasure
real_lr = LogisticRegression(max_iter=1000).fit(X, Y)
beta = torch.from_numpy(real_lr.coef_)
assert beta.norm(p=torch.inf) > 0.1

eraser = LeaceEraser.fit(X_t, Y_t)
X_ = eraser(X_t)

# But learns nothing after
null_lr = LogisticRegression(max_iter=1000, tol=0.0).fit(X_.numpy(), Y)
beta = torch.from_numpy(null_lr.coef_)
assert beta.norm(p=torch.inf) < 1e-4

In [ ]:
import pandas as pd
import pickle
df_19 = pd.read_csv("../data/cv-corpus-19.0-delta-2024-09-13/en/validated.tsv", delimiter="\t")
df_21 = pd.read_csv("../data/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", delimiter="\t")
df_17 = pd.read_csv('../data/cv-corpus-17.0-delta-2024-03-15/en/validated.tsv', delimiter="\t")

embeddings_19 = {}
with open('../data/cv-corpus-19.0-delta-2024-09-13/commonvoice_embeddings.pkl', 'rb') as file:
    embeddings_19 = pickle.load(file)
# embeddings_21 = {}
# with open('../data/cv-corpus-21.0-delta-2025-03-14/commonvoice_embeddings.pkl', 'rb') as file:
#     embeddings_21 = pickle.load(file)
# embeddings_17 = {}
# with open('../data/cv-corpus-17.0-delta-2024-03-15/commonvoice_embeddings.pkl', 'rb') as file:
#     embeddings_17 = pickle.load(file)

In [55]:
y_train= []
X_train= []
for idx, gender in enumerate(df_21.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_train.append(gender)
        X_train.append(embeddings_21[df_21.path.tolist()[idx].split(".")[0]])
for idx, gender in enumerate(df_17.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_train.append(gender)
        X_train.append(embeddings_17[df_17.path.tolist()[idx].split(".")[0]])
y_test= []
X_test= []
for idx, gender in enumerate(df_19.gender.tolist()):
    if gender in ["male_masculine", "female_feminine"]:
        y_test.append(gender)
        X_test.append(embeddings_19[df_19.path.tolist()[idx].split(".")[0]])

In [56]:
from sklearn.linear_model import LogisticRegression
import torch
from sklearn.metrics import accuracy_score

lr = LogisticRegression(max_iter=1000).fit(X_train, y_train)
beta = torch.from_numpy(lr.coef_)

beta.norm(p=torch.inf)

assert beta.norm(p=torch.inf) > 0.1
predict_train = lr.predict(X_train)
predict_test = lr.predict(X_test)
accuracy_score(y_train, predict_train), accuracy_score(y_test, predict_test)



(0.8334376956793988, 0.6551724137931034)

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier().fit(X_train, y_train)

predict_train = rf.predict(X_train)
predict_test = rf.predict(X_test)
accuracy_score(y_train, predict_train), accuracy_score(y_test, predict_test)



(1.0, 0.6781609195402298)

In [58]:
len(y_train), len(y_test)

(1597, 87)

In [ ]:
from concept_erasure import LeaceEraser
eraser = LeaceEraser.fit(torch.tensor(X_train), torch.tensor([0  if y == "female_feminine" else 1 for y in y_train]))
X_test_debaised = eraser(torch.tensor(X_test))
X_train_debaised = eraser(torch.tensor(X_train))
assert(X_train_debaised != X_train)
rf2 = RandomForestClassifier().fit(X_train_debaised, y_train)
lr2 = LogisticRegression(max_iter=1000, tol=0.0).fit(X_train_debaised, y_train)

rf_predict_train_debiased = rf2.predict(X_train_debaised)
rf_predict_test_debiased = rf2.predict(X_test_debaised)

lr_predict_train_debaised = lr2.predict(X_train_debaised)
lr_predict_test_debaised = lr2.predict(X_test_debaised)
print(accuracy_score(y_train, rf_predict_train_debiased), accuracy_score(y_test, rf_predict_test_debiased))
print(accuracy_score(y_train, lr_predict_train_debaised), accuracy_score(y_test, lr_predict_test_debaised))
beta = torch.from_numpy(lr.coef_)
print(beta.norm(p=torch.inf))

# but now use these against some sort of task and see if gendered accuracy changes -- try accent classification maybe?

1.0 0.6551724137931034
0.8196618659987477 0.6551724137931034
tensor(0.1488, dtype=torch.float64)
